$$
\begin{align*}
& \max_{x_i} & &  \sum_{i \in \{1\dots n\}} x_i\times l(i)  \nonumber \\
& \textrm{s.t.} & & \sum_{i \in \{1\dots n\}} i \times x_{i} = n,\\
& & &  x_{i} \geq 0\nonumber, \\
& & &  x_{i} \in \mathbb{N} \nonumber. \\
\end{align*} $$

In [10]:
def unbounded_knapsack_with_assignments(n, l_dict):
    # DP table: dp[k] = (max_value, {x_i assignments})
    dp = [(0, {}) for _ in range(n + 1)]
    
    for k in range(1, n + 1):
        for i in l_dict:
            if i <= k:
                current_value = dp[k - i][0] + l_dict[i]
                if current_value > dp[k][0]:
                    new_assignments = dp[k - i][1].copy()
                    new_assignments[i] = new_assignments.get(i, 0) + 1
                    dp[k] = (current_value, new_assignments)
    
    max_value, x_assignments = dp[n]
    
    # Fill zeros for items not used
    for i in l_dict:
        if i not in x_assignments:
            x_assignments[i] = 0
    
    return max_value, x_assignments

# Exemplo de uso
n = 4
l_dict = {1: 4, 2: 8, 3: 13, 4: 15}

max_value, x_opt = unbounded_knapsack_with_assignments(n, l_dict)

print("Optimal x_i assignments:")
for i in sorted(x_opt.keys()):
    print(f"x_{i} = {x_opt[i]}")
print(f"Maximum objective value: {max_value}")

Optimal x_i assignments:
x_1 = 1
x_2 = 0
x_3 = 1
x_4 = 0
Maximum objective value: 17
